# Purpose:

I want to model my allele frequencies and ultimately infer the SD and SZ parameters. The first thing I can do to model this is to figure out what the uncertainty is in the allele frequencies. This is going to be based on the sequences of sampling. Then I can use this to at last use simple curve fitting to generate better SD parameters. The tough part is thinking about how SZ parameter fits in this.

## Modelling allele frequencies:

There are several sampling processes that are involved in determining the error of determing the underlying allele frequencies. First, let's set up our experiment in simple terms: We have a cross between P1/P1 and P1/P2 and we are going to sample N amount of progeny. We want to know what the frequency of P2 will be in this next generation. And more concretely we want to know what the underlying distribution that generates the allele frequency is. In other words, if a certain locus has an empirical AF of .2 we want to know if this reflects the true distribution or is just noise.

### Meiosis
The first sampling process is a biological process. It is the meiosis and generation of progeny. In this process the probability of sampling a P1, or P2 chromosome is a simple Bernouli trial, or binomial process:

$Bin \sim (p)$

Where p is the probability of drawing a P2 chromosome. This probability is shifted however when we consider drive. It then becomes:

$Bin \sim (p+D)$ 

Where D is the strength of the drive locus. For now let's ignore this. In a binomial process the variance is simply $npq$, so let's keep that in mind.

### Sampling individuals:

The second sampling process is to sample cells/individuals from our cohort of progeny. For example we have 4,000 flies that were grown and we now are going to sample ~2,000 cells from this group. The probability of sampling a cell from a given fly is essentially a sampling without replacement -- a multinomial probability distribution:

$MultiN \sim (\pi_i)$

Where each individual has a $\pi_i$ parameter associated with the probability of sampling a cell from that individual. The $\pi$ parameter in this pmf is a metric of how likely an individual is to be sampled, which is representative of body size differences of the flies. By this I mean that larger flies will be sampled more than smaller flies and would have a bigger $\pi$. The body size differences are most appropriately modelled by a normal distribution:

$N \sim (\mu, \sigma^2)$

Which means that each $\pi_i$ is drawn from a normal distribution. Consequently, genotype dependent body size differences will generate two normal distributions with means that are shifted with respect to each other. This is an important thing I wish to get at, but for the simple first pass we ought to ignore size and segregation distortion.

### Sampling reads:

The third sampling process is the actual sequencing. In this process we know that sampling an allele from a given cell is also multinomial process:

$MultiN \sim (\pi_{P1}, \pi_{P2}, \pi_E)$

The probability of sampling a P1, or P2 allele is dependent on whether or not the segment sampled is from a P1/P1 segment, or a P1/P2 segment. The probability of homozygosity or heterozygosity is modelled explicitly by the HMM. 


## Goal:

The goal of the analysis is to know the underlying binomial distribution, specifically $p$. But our estimate of this parameter is dependent on both sampling of the cells and sampling of the $p$ from the cross. 

First I am going to try to familiarize myself with pystan to estimate $p$ from a binomial random sample.

In [2]:
import pystan
import numpy as np

In [10]:
model = """
data {
    int<lower=0> N; // number of F1
    vector[N] y; // bernoulli random variable

}
parameters {
    real p;
    
}

model {
    
    y ~ binom(p);
}
"""

schools_dat = {'N': 1000,
               'y': np.random.binomial(n=1, p=.6, size=1000),
               }


In [11]:
sm = pystan.StanModel(model_code=model)
fit = sm.sampling(data=schools_dat, iter=1000, chains=4)

ValueError: Failed to parse Stan model 'anon_model_530355a74ced3ee584881cc7a68d49ca'. Error message:
SYNTAX ERROR, MESSAGE(S) FROM PARSER:
Probability function must end in _lpdf or _lpmf. Found distribution family = binom with no corresponding probability function binom_lpdf, binom_lpmf, or binom_log
 error in 'unknown file name' at line 14, column 17
  -------------------------------------------------
    12: model {
    13:     
    14:     y ~ binom(p);
                        ^
    15: }
  -------------------------------------------------

